In [ ]:
import pandas as pd
import configparser
import openai

# Read credentials
parser = configparser.ConfigParser()
parser.read('./config/config.ini')
print(parser['openai']['TRAINING_DATA_FILE_ID'])

# Read raw dataset
# Change the iteration folder number for every iteration
data = pd.read_csv("./dataset/raw-data.csv")
data_df = pd.DataFrame(data)

data_df

In [ ]:
# Convent CSV to JSONL
data_df.to_json("./dataset/training-data.jsonl", orient='records', lines=True)

In [ ]:
# Add OpenAI credentials
openai.api_key = parser['openai']['ACCESS_KEY']

In [ ]:
# Lists the fine tune odels created
openai.FineTune.list()

In [ ]:
# Sublets the dataset into training and validation jsonl
# Formats the dataset for best training results
# Run in terminal

cd <project folder>/packages/classifer/dataset
openai tools fine_tunes.prepare_data -f 'training-data.jsonl' -q

In [ ]:
# Upload training and validation jsonl file
# Retrieve the ID
# Run this code once to upload the file

training_data = openai.File.create(
  file=open("./dataset/training-data_prepared_train.jsonl", "rb"),
  purpose='fine-tune'
)
training_data_file_id = training_data.id

validation_data = openai.File.create(
  file=open("./dataset/training-data_prepared_valid.jsonl", "rb"),
  purpose='fine-tune'
)
validation_data_file_id = validation_data.id

In [ ]:
print(training_data_file_id)
print(validation_data_file_id)

In [ ]:
# Snippet to delete the file

training_data_file_id = parser['openai']['TRAINING_DATA_FILE_ID']
validation_data_file_id = parser['openai']['VALIDATION_DATA_FILE_ID']

openai.File.delete(training_data_file_id)
openai.File.delete(validation_data_file_id)

In [ ]:
# Equivalent CLI query: openai api fine_tunes.create -t "training-data_prepared_train.jsonl" -v "training-data_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 7

openai.FineTune.create(
  training_file=parser['openai']['TRAINING_DATA_FILE_ID'],
  validation_file=parser['openai']['VALIDATION_DATA_FILE_ID'],
  model=parser['openai']['MODEL_NAME'],
  compute_classification_metrics=True,
  classification_n_classes=7,
  suffix="ayra"
)

In [ ]:
openai.FineTune.retrieve(id='ft-O7RMjF3IiBatt5YDf2YY2xqv')

In [ ]:
# Checks the status of the fine tune job

openai.FineTune.list_events(id="ft-O7RMjF3IiBatt5YDf2YY2xqv")

# Command Line version

# openai --api-key <api-key> api fine_tunes.follow -i <fine-tune_id>